Есть финансовый отчет о затратах в детализаии регион-бизнес, обновляемый ежемесячно. Данные из 1С предприятие и других источников, в том числе подготовленных вручную. Количество регионов 15, количество бизнесов 5.

В отчете отражены фактические продажи в рублях и физических показателях - штуки, объем, вес, линии. Линия - это ассортиментная единица товара.

Затраты представлены группами - затраты на сборку товара на складе по заказам покупателей, транспортные затраты, аренда помещений, эксплуатация помещений, затраты на упаковку, фонд оплаты труда, прочие операционные и прочие внереализационные расходы.

Требуется проверять отчет на наличие ошибок, возникающих из-за неполной загрузки данных, а также из-за неверно предоставленной информации в файлах, подготовленных вручную.

Идея автоматизации проверки в том, чтобы выявить скореллированные показатели за период 12 месяев назад от проверяемого, посчитать относительные показатели, например, рублевую выручку поделить на  проданные штуки, и по этим относительным показателям посчитать расстояние до среднего значения этого временного ряда из относительных показателей в стандартных отклонениях. Если расстояние до среднего превысит три стандартных отклонения, то имеет смысл проверить данные на наличие ошибки.



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#N - строгость проверки (размер отклонения от среднего в сигмах)
N = 3

In [ ]:
#считываем затраты и показатели, соединяем в один фрейм
df_z = pd.read_excel('Сводный.xlsx')
df_q = pd.read_excel('Показатели.xlsx')


In [ ]:
df = df_z.merge(df_q, on=['Month','Region','Business'], how='inner')
df = df[(df['Month'] >= '2023-01-01')].sort_values(by  = ['Region','Month'])

In [ ]:
#считаем суммарный товарооборот для удобстав представления данных
df.Sales_NDS = df.Sales_NDS + df.Sales_3PL_NDS

df = df.drop(['Sales_3PL_NDS'],axis = 'columns')
#df.head()

In [ ]:
#создаем список бизнесов и регионов
Business = pd.unique(df['Business'].tolist())
Region = pd.unique(df['Region'].tolist())

In [ ]:
#матрица МОДУЛЕЙ коэфф. корреляции - для определения основных и влияющих показателей
df_corr = df.corr(numeric_only = True).abs()


Корреляционная матрица. По ней мы будем судить о том, зависимы ли показатели друг от друга.

In [ ]:
df_corr.head()

,Unit_sold,Volume_sold,Sales_NDS,Sborka,Transport_Client,Transport_Region,Transport_Provider,Marketplaces,Еstate_rent,Exploitation,Рackaging,FOT,Operating_expenses,Non_operating_ expenses,Line,Unit,Box,Volume,Weight
Unit_sold,1.000000,0.997158,0.964439,0.924247,0.936747,0.340065,0.939883,0.061712,0.845214,0.804472,0.948240,0.049853,0.911428,0.472312,0.926425,0.979654,0.982584,0.981462,0.979250
Volume_sold,0.997158,1.000000,0.974977,0.920928,0.931699,0.323265,0.952500,0.055622,0.831625,0.806901,0.953005,0.044649,0.911480,0.470842,0.922648,0.975724,0.981777,0.982308,0.981667
Sales_NDS,0.964439,0.974977,1.000000,0.915936,0.929183,0.315100,0.991658,0.037031,0.804664,0.811365,0.951811,0.067373,0.934186,0.473684,0.927349,0.954151,0.960131,0.966520,0.970670
Sborka,0.924247,0.920928,0.915936,1.000000,0.969597,0.535903,0.897604,0.035956,0.933627,0.819567,0.914317,0.040514,0.964835,0.436626,0.953431,0.942583,0.938366,0.940427,0.939488
Transport_Client,0.936747,0.931699,0.929183,0.969597,1.000000,0.533325,0.915666,0.021199,0.939542,0.815547,0.930720,0.036683,0.972834,0.459069,0.969519,0.961051,0.953068,0.955622,0.953145


Функция приводит все числовые значения таблицы к единому масштабу методом Минимакс. Цель — преобразовать данные в диапазон [0..1],  чтобы все числовые признаки были равны в возможностях своего влияния друг на друга.

Затем высчитываются соотношения основного параметра-признака и влияющих на него параметров. Это относительные показатели. Геометрический смысл относительного показателя - тангенс угла наклона прямой соединяющей начало координат с точкой данных (координаты точки -  это пара скореллированных показателей) и положительным направлением оси абсцисс.

Если коэффииент корреляции двух показателей близок к единице, то точки на плоскости лежат вдоль прямой с конкретным углом наклона.

Отклонение от этого угла мы и будем "ловить", это значит, что показатели рассинхронизированы, что-то "пошло не так", возможно выгружены не все данные или неверно отражены финансовые затраты.

In [ ]:
#функция подсчета z оценок для каждого фрейма, главного параметра и влияющих на него параметров


def estimation(df,main_parameter,Рarameters):

    #отделяем числовую часть данных для нормализации
    df_numbers = df.iloc[:,3:]
    #сохраняем текстовую часть фрейма
    df_head = df.iloc[:,0:3].reset_index()
    #нормализация числовой части
    from sklearn import preprocessing
    scaler = preprocessing.MinMaxScaler()
    names = df_numbers.columns
    d = scaler.fit_transform(df_numbers)
    scaled_df = pd.DataFrame(d, columns=names)
    #собираем нормализованные и текстовые данные построчно справа
    scaled_df = pd.concat([df_head, scaled_df],axis = 1)

    #Перебираем все параметры для нормализованного фрейма данных
    #Внутри каждого бизнеса и каждого региона считаем ОТНОШЕНИЕ основного показателя к влияющему параметру
    #Методом z оценок ищем аномалии в созданном ряду чисел

    df_new = pd.DataFrame()#пустой фрейм куда будем складывать строки с максимальными отклонениями


    for parameter in Рarameters:

        for  region in Region:

             for  business in Business:

                  scaled_df_temp = scaled_df[(scaled_df['Region'] == region) & (scaled_df['Business'] == business)]

                  if scaled_df_temp.shape[0] > 6: #для датасетов больше шести мес ищем z-оценки, иначе не накоплена статистика, не с чем сравнить

                    #отношение скоррелированных параметров = тангенс угла наклона прямой регрессии
                    scaled_df_temp = scaled_df_temp.assign(fraction = lambda x: x[main_parameter] / x[parameter])

                    scaled_df_temp = scaled_df_temp.fillna(0) #запоняю пустоту нулями
                    scaled_df_temp.replace([np.inf , -np.inf ], 0 , inplace= True ) #если было деление на ноль, то меняю ошибку на число ноль

                    #z оценки тангенсов угла наклона - то есть относительных показателей
                    scaled_df_temp['zscore'] = stats.zscore(scaled_df_temp['fraction'])

                    scaled_df_temp = scaled_df_temp[['Month','Region','Business','zscore']] #оставляю только нужные колонки

                    scaled_df_temp['parametr2'] = parameter #наименование влияющего параметра мне тоже нужно


                    df_new = pd.concat([df_new, scaled_df_temp],axis = 0) #сохраняю результат работы

                  else:
                     continue #датасеты  с недостаточным количеством данных пропускаем, они возникают когда появился новый регион или новый бизнес



    df_new = df_new[(df_new['zscore'] > N)] #оставляем только значимые строки, где расстояние до среднего больше N сигм и имеет смысл проверять на наличие ошибок

    return(df_new)

In [ ]:
df_result = pd.DataFrame()#пустой фрейм куда будем складывать результаты работы функции на разных парах параметров

Берем первый физический показатель "Продано, шт." и по Корреляционной матрице выбираем для него влияющие показатели, критерий - чтобы Коэффициент корреляции был близко к 1, именно поэтому не взяли  затраты ФОТ, услуги маркетплейсов, прочие внереализаионные расходы  -  там коэф = 0,47 и меньше.

Применяем к полученной таблице функцию поиска ошибок и сохраняем результат.

In [ ]:
#выбор зависимых показателей для "Продано, шт." по первой строке матрицы корреляции

df_unit_sold = df[['Month','Region','Business','Unit_sold',
         'Sales_NDS',
         'Sborka',
         'Transport_Client',
         'Transport_Provider',
         'Еstate_rent',
         'Operating_expenses',
         'Line',
         'Unit',
         'Box',
         'Volume',
         'Weight' ]]


#создаем список зависимых параметров для "Продано,шт"
Рarameters_unit_sold = df_unit_sold.columns.tolist()[3:]

#Применили функцию к "Продано,шт" и сохранили во временный фрейм
df_temp = estimation(df_unit_sold, 'Unit_sold', Рarameters_unit_sold)

#Добавили столбец с названием "Продано,шт" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Unit_sold"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

Берем физический показатель "Продано, м3" и подбираем для него скореллированные параметры. C каждым новым показателм список будет уменьшаться, тк нам не нужны дубликаты пар показателей, которые мы уже проверяли на предыдущем шаге.

In [ ]:
#выбор зависимых показателей для "Продано, м3" по 2 строке матрицы корреляции

df_volume_sold = df[['Month','Region','Business','Volume_sold',
         'Sales_NDS',
         'Sborka',
         'Transport_Client',
         'Transport_Provider',
         'Еstate_rent',
         'Operating_expenses',
         'Line',
         'Unit',
         'Box',
         'Volume',
         'Weight']]

#создаем список зависимых параметров для "Продано,м3"
Рarameters_volume_sold = df_volume_sold.columns.tolist()[3:]

#Применили функцию к "Продано,м3" и сохранили во временный фрейм
df_temp = estimation(df_volume_sold,"Volume_sold", Рarameters_volume_sold)

#Добавили столбец с названием "Продано,м3" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Volume_sold"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Продажи с НДС" по 3 строке матрицы корреляции

df_sales_nds = df[['Month','Region','Business','Sales_NDS',
                  'Sborka',
                  'Transport_Client',
                  'Transport_Provider',
                  'Еstate_rent',
                  'Operating_expenses',
                  'Line',
                  'Unit',
                  'Box',
                  'Volume',
                  'Weight']]

#создаем список зависимых параметров для "Продано,м3"
Рarameters_sales_nds = df_sales_nds.columns.tolist()[3:]

#Применили функцию к "Продано,м3" и сохранили во временный фрейм
df_temp = estimation(df_sales_nds,"Sales_NDS", Рarameters_sales_nds)

#Добавили столбец с названием "Продажи с НДС" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Sales_NDS"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Стоимость сборки" по 4 строке матрицы корреляции

df_sborka = df[['Month','Region','Business','Sborka',
                  'Transport_Client',
                  'Transport_Provider',
                  'Еstate_rent',
                  'Operating_expenses',
                  'Line',
                  'Unit',
                  'Box',
                  'Volume',
                  'Weight']]

#создаем список зависимых параметров для "Продано,м3"
Рarameters_sborka = df_sborka.columns.tolist()[3:]

#Применили функцию к "Стоимость сборки" и сохранили во временный фрейм
df_temp = estimation(df_sborka,"Sborka", Рarameters_sborka)

#Добавили столбец с названием "Продажи с НДС" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Sborka"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Транспорт до клиента" по 5 строке матрицы корреляции

df_transport_client = df[['Month','Region','Business','Transport_Client',
                          'Transport_Provider',
                          'Еstate_rent',
                          'Operating_expenses',
                          'Line',
                          'Unit',
                          'Box',
                          'Volume',
                          'Weight']]

#создаем список зависимых параметров для "Транспорт до клиента"
Рarameters_transport_client = df_transport_client.columns.tolist()[3:]

#Применили функцию к "Транспорт до клиента" и сохранили во временный фрейм
df_temp = estimation(df_transport_client,"Transport_Client", Рarameters_transport_client)

#Добавили столбец с названием "Продажи с НДС" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Transport_Client"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Транспорт от поставщика" по 7 строке матрицы корреляции

df_transport_provider = df[['Month','Region','Business','Transport_Provider',
                            'Еstate_rent',
                            'Operating_expenses',
                            'Line',
                            'Unit',
                            'Box',
                            'Volume',
                            'Weight']]

#создаем список зависимых параметров для "Транспорт от поставщика"
Рarameters_transport_provider = df_transport_provider.columns.tolist()[3:]

#Применили функцию к "Транспорт от поставщика" и сохранили во временный фрейм
df_temp = estimation(df_transport_provider,"Transport_Provider", Рarameters_transport_provider)

#Добавили столбец с названием "Транспорт от поставщика" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Transport_Provider"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Затраты на аренду" по 9 строке матрицы корреляции

df_estate_rent = df[['Month','Region','Business','Еstate_rent',
                            'Operating_expenses',
                            'Line']]

#создаем список зависимых параметров для "Транспорт от поставщика"
Рarameters_estate_rent = df_estate_rent.columns.tolist()[3:]

#Применили функцию к "Транспорт от поставщика" и сохранили во временный фрейм
df_temp = estimation(df_estate_rent,"Еstate_rent", Рarameters_estate_rent)

#Добавили столбец с названием "Транспорт от поставщика" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Еstate_rent"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Операционные расходы" по 13 строке матрицы корреляции

df_operating_expenses = df[['Month','Region','Business','Operating_expenses',
                            'Line',
                            'Unit',
                            'Box',
                            'Volume',
                            'Weight']]

#создаем список зависимых параметров для "Операционные расходы"
Рarameters_operating_expenses = df_operating_expenses.columns.tolist()[3:]

#Применили функцию к "Операционные расходы" и сохранили во временный фрейм
df_temp = estimation(df_operating_expenses,"Operating_expenses", Рarameters_operating_expenses)

#Добавили столбец с названием "Операционные расходы" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Operating_expenses"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

In [ ]:
#выбор зависимых показателей для "Линии" по 15 строке матрицы корреляции

df_line = df[['Month','Region','Business','Line',
                            'Unit',
                            'Box',
                            'Volume',
                            'Weight']]

#создаем список зависимых параметров для "Линии"
Рarameters_line = df_line.columns.tolist()[3:]

#Применили функцию к "Линии" и сохранили во временный фрейм
df_temp = estimation(df_line,"Line", Рarameters_line)

#Добавили столбец с названием "Линии" и сохранили в накопительный итоговый файл
df_temp['parametr1'] = "Line"

#сохранили результат работы функции в накопительный фрейм
df_result = pd.concat([df_result,df_temp],axis = 0)

Теперь у нас есть полный список всех проверенных пар показателей, с отобранными значениями, "подозреваемыми на ошибки" за полный период 12 месяцев. Но нам нужны только данные проверяемного месяца, так как предполагается что в прошлых периодах все проверено и исправлено, и если есть необычные значения, то они не являются ошибочными.

In [ ]:
#сортируем по убыванию оценок
#оставляем только последний месяц
MaxMonth = df_result['Month'].max()
df_result = df_result[(df_result['Month'] == MaxMonth) ]

df_result = df_result.sort_values(by = 'zscore',ascending = False)

Результат - список регионов\бизнесов\пар показателей, которые показали необычное соотношение друг к другу в проверяемом периоде. Например, Спб\ бюджетный рынок\ количество проданных штук и затраты на сборку нужно проверить, возможно недогружены данные или неверно отражены затраты.

Повторяющиеся пары регион\бизнес говорят не только об ошибке, но и, как вариант о том, что продана необычная партия товара, например очень крупная, по которой все затраты существенно меньше чем в прошлых периодах из-за оптовых условий отгрузки.

То есть не все "подозреваемые" действительно оказываются ошибками.
Но таким образом были выявлены скрытые ошибки, когда в системе 1С Предприятие были сделаны изменения, о которых никто не узнал своевременно, а именно - физические показатели продаж начали считаться по другому алгоритму при сохранении порядка затрат. Метод показал рассинхрон показателей с затратами и ошибка была исправлена.

In [ ]:
df_result.head(20)

,Month,Region,Business,zscore,parametr2,parametr1
586,2024-03-01,Санкт-Петербург,Дистрибуция на бюджетном рынке,3.453743,Unit,Sborka
586,2024-03-01,Санкт-Петербург,Дистрибуция на бюджетном рынке,3.286474,Unit,Transport_Provider
586,2024-03-01,Санкт-Петербург,Дистрибуция на бюджетном рынке,3.110857,Unit,Volume_sold
586,2024-03-01,Санкт-Петербург,Дистрибуция на бюджетном рынке,3.050419,Volume,Sborka
586,2024-03-01,Санкт-Петербург,Дистрибуция на бюджетном рынке,3.036408,Line,Sborka
